In [1]:
import requests
import json
import codecs
from bs4 import BeautifulSoup
import pandas as pd

In [47]:
%matplotlib inline

In [2]:
api_uri ='https://sp.olx.com.br/vale-do-paraiba-e-litoral-norte/vale-do-paraiba/autos-e-pecas/carros-vans-e-utilitarios'

In [3]:
dummy_product_names = []
product_names = []

In [58]:
def toint(x):
    try:
        return int(x)
    except:
        return 0
def get_km(x):
    try:
        return int(x.split('km |')[0].replace('.', ''))
    except:
        return 0
def get_dados(lista):
    dados = []
    for label in lista.find_all('li'):
        nome, price, desc, ano, fotos, km = '', '', '', '','', ''
        item = label.find('h2', {'class': 'OLXad-list-title'})
        if item:
            nome = item.text.strip()
            if nome:
                ano = nome.split('-')[-1]
        item = label.find('p', {'class': 'OLXad-list-price'})
        if item:
            price = item.text.strip().replace('R$', '').replace('.', '')
        item = label.find('p', {'class': 'text detail-specific'})
        if item:
            desc = item.text.strip().replace('\n', '').replace('\t', '')
            km = get_km(desc)
        item = label.find('div', {'class': 'OLXad-list-image-counter'})
        if item:
            d = item.text.strip().split('\t')
            fotos = d[-1] if d else 0
        dados.append([nome, price, desc, ano, fotos, km])
    return dados   

In [27]:
url = 'https://sp.olx.com.br/vale-do-paraiba-e-litoral-norte/vale-do-paraiba/autos-e-pecas/carros-vans-e-utilitarios'

In [40]:
lista_carros = []
for page in range(101):
    conteudo = requests.get(url, params={'o': page}, headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:55.0) Gecko/20100101 Firefox/55.0"}).content
    soup = BeautifulSoup(conteudo, "html.parser")
    lista = soup.find('ul', {'id': 'main-ad-list'})
    lista_carros.extend(get_dados(lista))

In [116]:
carros = pd.DataFrame(lista_carros, columns=['nome', 'price', 'desc', 'ano', 'fotos', 'km'])

In [117]:
carros.shape

(5454, 6)

In [118]:
carros['price'] = carros['price'].apply(toint)
carros['ano'] = carros['ano'].apply(toint)
carros['fotos'] = carros['fotos'].apply(toint)
carros['km'] = carros['km'].apply(toint)

In [119]:
carros.to_csv('/tmp/carros.csv', index=False)

In [120]:
carros[(carros.ano==2014) & (carros.nome.str.lower().str.contains('hb20s'))].sort_values('price')

,nome,price,desc,ano,fotos,km
249,(OP) Hyundai Hb20s 1.6 - Completo - Baixo km -...,0,0 km | Câmbio: automático | Flex,2014,5,0
1531,/Hyundai Hb20s Premium 1.6 - Completo + Multim...,0,0 km,2014,5,0
2421,R HB20S sedã 2014 Completo - 2014,0,0 km | Câmbio: manual | Flex,2014,3,0
3617,Hyundai Hb20s 1.0 Comfort Style 12v - 2014,33990,71.147 km | Câmbio: manual | Flex,2014,8,71147
4729,Vendo HB20S Confort Plus 1.0 top - Analiso Tro...,34500,80 km | Câmbio: manual | Flex,2014,3,80
3637,Hb20S confort plus 1.0 flex 2014 completo - 2014,36900,116.000 km | Câmbio: manual | Flex,2014,18,116000
4769,Hyundai Hb20s confort plus 1.0 flex completo i...,36900,108.000 km | Câmbio: manual | Flex,2014,18,108000
893,HB20S 1.6 2014 Completo Abx Tabela - 2014,38000,82.000 kmmanualFlex,2014,10,0
392,Hyundai HB20S Comfort Plus 1.6 - 2014,39500,85.000 kmmanualFlex,2014,5,0
3709,HB20s 1.6 2014 - 2014,39900,102.821 km | Câmbio: manual | Flex,2014,6,102821
